In [ ]:
!env\Scripts\activate.bat

In [ ]:
!pip list

In [ ]:
#from dask.distributed import Client
#client = Client(scheduler_file=None)
#print(client)

In [ ]:

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from dask.distributed import Client
client = Client(n_workers=4, threads_per_worker=2, memory_limit='6GB')

In [ ]:
client

In [ ]:
#client.close()

In [ ]:
import dask.dataframe as dask
import dask_ml

In [ ]:

df = dask.read_csv('male_players.csv', dtype={'cf': 'object',
                                    'club_contract_valid_until_year': 'float64',
                                    'club_jersey_number': 'float64',
                                    'club_loaned_from': 'object',
                                    'club_team_id': 'float64',
                                    'league_id': 'float64',
                                    'league_level': 'float64',
                                    'lf': 'object',
                                    'lw': 'object',
                                    'release_clause_eur': 'float64',
                                    'rf': 'object',
                                    'rw': 'object',
                                    'value_eur': 'float64',
                                    'wage_eur': 'float64'},
                   assume_missing=True)

In [ ]:
df.columns

In [ ]:

numeric_cols = [col for col, dtype in df.dtypes.items() if dtype.kind in ['f', 'i']]
df_numeric = df[numeric_cols]

In [ ]:
string_columns = [col for col, dtype in df.dtypes.items() if dtype.kind in ['O']]
df_string = df[string_columns]
df_string["international_reputation"] = df_numeric["international_reputation"].astype('float64')
df_string['age']=df_numeric['age'].astype('float64')

In [ ]:
df_string.head(10)

In [ ]:
#num_rows = len(df)
#print(f"El DataFrame tiene {num_rows} filas.")
print("El DataFrame tiene 10003590 filas.")

In [ ]:
print(df['short_name'].head(10,npartitions=2))

In [ ]:
print(df.head(10,npartitions=2))

In [ ]:
print(df.info()) 

In [ ]:
print(df_numeric.head(10,npartitions=1))

In [ ]:
df_cleaned = df_numeric.map_partitions(lambda x: x.fillna(0))

In [ ]:

print(df_cleaned.head(10,npartitions=1))

In [ ]:
n_rows = 10000
df_subset=df_cleaned.head(n_rows)
df_subset_string=df_string.head(n_rows)

In [ ]:
df[["long_name","age"]].head(5)

In [ ]:
df_subset_string.head(5)

In [ ]:
df_subset.head(5)

In [ ]:
df_subset.columns

In [ ]:
df_subset.head(10)

In [ ]:
df_subset = df_subset.drop(columns=['club_jersey_number'])

In [ ]:
df_y=df_subset['international_reputation']
df_y.head(10)

In [ ]:
#eliminar columnas especificas
df_subset = df_subset.drop(['player_id','value_eur','fifa_update',
                            'goalkeeping_speed','fifa_version',
                            'release_clause_eur','wage_eur',
                            'club_contract_valid_until_year',
                            'league_id','club_team_id','nationality_id',
                            'nation_team_id','international_reputation','age',
                            'league_level', 'nation_jersey_number', 'weak_foot',
                            'pace', 'passing', 'dribbling', 'physic',
                            'attacking_crossing', 'attacking_finishing',
                            'attacking_heading_accuracy', 'attacking_short_passing',
                            'attacking_volleys', 'skill_dribbling', 'skill_curve',
                            'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
                            'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
                            'movement_reactions', 'movement_balance', 'power_shot_power',
                            'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
                            'mentality_aggression', 'mentality_interceptions',
                            'mentality_positioning', 'mentality_vision', 'mentality_penalties',
                            'mentality_composure', 'defending_marking_awareness',
                            'defending_standing_tackle', 'defending_sliding_tackle',
                            'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
                            'goalkeeping_positioning', 'goalkeeping_reflexes'], axis=1)

In [ ]:
df_subset.head(5)

In [ ]:
len(df_y.unique())

In [ ]:
from dask_ml.cluster import KMeans

kmeans = KMeans(n_clusters=len(df_y.unique()), random_state=0)


In [ ]:
import dask.array as da
X_array = da.from_array(df_subset.to_numpy())

In [ ]:
kmeans.fit(X_array)

In [ ]:
#for chunk in df_subset.to_dask_array(lengths=True):
#    kmeans.partial_fit(chunk)
#    for i, inertia in enumerate(kmeans.inertia_):
#        print(f"Iteration {i}: inertia={inertia}")

In [ ]:
import matplotlib.pyplot as plt

labels = kmeans.labels_
print(labels)

In [ ]:
df_subset.columns

### AGRUPACION SEGUN EL PROMEDIO DEL JUGADOR Y SU SUELDO

In [ ]:

fig, ax = plt.subplots()
scatter = ax.scatter(df_subset['skill_moves'], df_subset['overall'], c=labels, cmap='viridis')
legend = ax.legend(*scatter.legend_elements(),
                    loc="upper right", title="Clusters")
ax.add_artist(legend)
plt.show()

### AGRUPACION SEGUN EL PROMEDIO DEL JUGADOR Y SU SUELDO

In [ ]:

fig, ax = plt.subplots()
scatter = ax.scatter(df_subset['defending'], df_subset['potential'], c=labels, cmap='viridis')
legend = ax.legend(*scatter.legend_elements(),
                    loc="upper right", title="Clusters")
ax.add_artist(legend)
plt.show()

In [ ]:

fig, ax = plt.subplots()
scatter = ax.scatter(df_subset['shooting'], df_subset['potential'], c=labels, cmap='viridis')
legend = ax.legend(*scatter.legend_elements(),
                    loc="upper right", title="Clusters")
ax.add_artist(legend)
plt.show()

In [ ]:
#prediccion 
y_pred=kmeans.predict(X_array)
y_pred_computed, = dask.compute(y_pred)
y_pred_computed

In [ ]:
labels = labels.compute()
print(labels)

In [ ]:
from dask_ml.metrics import accuracy_score

In [ ]:
from dask_ml.model_selection import train_test_split

In [ ]:

from sklearn.ensemble import RandomForestClassifier


In [ ]:

# 4. Entrenar el modelo
X_train, X_test, y_train, y_test = train_test_split(X_array, df_y)
X_train


In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

In [ ]:
# 5. predicciones
y_pred = clf.predict(X_test)
y_pred

In [ ]:
y_test_array = da.from_array(y_test.values)
y_test_array

In [ ]:
y_pred_array = da.from_array(y_pred)
y_pred_array

In [ ]:
accuracy = accuracy_score(y_test_array, y_pred_array)
accuracy

# QUE JUGADOR SOY?

In [ ]:
diccionario={'overall':[91.0],
             'potential':[91.0],
             'height_cm':[185.0],
             'weight_kg':[80.0],
             'skill_moves':[4.0],
             'shooting':[92.0],
             'defending':[33.0]
             }
diccionario

In [ ]:
import dask.dataframe as dd
df_nuevo=dd.from_dict(diccionario,orient='columns',npartitions=1)
df_nuevo.values

In [ ]:
y_pred_nuevo=clf.predict(df_nuevo.values)
resultado=float(y_pred_nuevo[0])
resultado

#### BUSCANDO EL JUGADOR QUE SEA NIVEL SIMILAR A MIS HABILIDADES

In [ ]:
df_subset_string['international_reputation']

In [ ]:
import numpy as np
fila_aleatoria = df_subset_string.query('international_reputation == 5').sample(frac=1, random_state=np.random.randint(len(df_subset_string))).head(1)
fila_aleatoria

In [ ]:
#obtener las columnas de la fila_aleatoria desde la coluumna 10 hasta la 20
fila_aleatoria.iloc[:,10:20]

In [ ]:
imagen=fila_aleatoria["player_face_url"]
nombre=fila_aleatoria["short_name"]
nombre_completo=fila_aleatoria["long_name"]
club_actual=fila_aleatoria["club_name"]
liga_actual=fila_aleatoria["league_name"]
pierna=fila_aleatoria["preferred_foot"]
nacionalidad=fila_aleatoria["nationality_name"]
nacimiento=fila_aleatoria["dob"]
edad=fila_aleatoria["age"]

In [ ]:
nombre=nombre.values[0]
nombre_completo=nombre_completo.values[0]
club_actual=club_actual.values[0]
liga_actual=liga_actual.values[0]
pierna=pierna.values[0]
nacionalidad=nacionalidad.values[0]
nacimiento=nacimiento.values[0]
edad=edad.values[0]
nombre,nombre_completo,club_actual,liga_actual,pierna,nacionalidad,nacimiento,edad

In [ ]:
imagen=imagen.values[0]
imagen

In [ ]:
from PIL import Image
import requests
from io import BytesIO
from IPython.display import display, Image as IPImage

In [ ]:
response=requests.get(imagen)
img=Image.open(BytesIO(response.content))
display(IPImage(data=response.content))
print(f"TE PARECES AL JUGADOR: {nombre_completo} \n")
print(f"Es mas conocido como : {nombre} \n")
print(f"Juega en el club: {club_actual} \n")
print(f"Juega en la liga: {liga_actual} \n")
print(f"Es de nacionalidad: {nacionalidad} \n")
print(f"Su fecha de nacimiento es: {nacimiento} \n")
print(f"Su edad es: {int(edad)} años\n")
if (pierna=="Right"):
    print(f"Es diestro \n")
else:
    print(f"Es zurdo \n")

In [ ]:
client.close()  # close the client